In [ ]:
!pip install tpot
from sklearn.metrics import accuracy_score

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.datasets import load_boston
 
 
import pandas as pd #this is how we usually import pandas
import numpy as np #this is how we usually import numpy
 
import matplotlib #only needed to determine Matplotlib version number
#import tables # pytables is needed to read and write hdf5 files
#import openpyxl # is used to read and write MS Excel files
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
 
import xgboost
 
import warnings
warnings.filterwarnings('ignore')

imports

In [ ]:

data = pd.read_csv('/content/drive/MyDrive/Finalfinal.csv')

data = data.drop('SubjectID' , axis=1)


#if data contains sex feature
#size_mapping = {
     #   #   'M': 0,
       #    'F': 1,
        #         }
#data['Sex'] = data['Sex'].map(size_mapping)


features_all = data.drop('Age', axis=1)
target = data['Age']
target = target.values.reshape(-1,1)

In [ ]:
features_all.shape , target.shape

# **split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_all, target,
                                                    train_size=0.9, test_size=0.1, random_state=42)

# **feature selection**

#filter methods

 Remove Constant Features

In [ ]:
# remove constant features


constant_features = [feat for feat in X_train.columns if X_train[feat].std() == 0]

 
X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)
 
 
X_train.shape, X_test.shape

Remove quasi-constant features

In [ ]:
# Remove quasi-constant features
sel = VarianceThreshold(threshold=0.01)  # 0.1 indicates 99% of observations approximately
sel.fit(X_train)  # fit finds the features with low variance
sum(sel.get_support()) # how many not quasi-constant?


# Get the remaining features
features_to_keep = X_train.columns[sel.get_support()]


# Remove the constant features
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)
 


# Transform the arrays back to Dataframes
X_train= pd.DataFrame(X_train)
X_train.columns = features_to_keep
 
X_test= pd.DataFrame(X_test)
X_test.columns = features_to_keep

X_train.shape, X_test.shape

Remove duplicated features

In [ ]:
# Check for duplicated features in the training set
duplicated_feat = []
for i in range(0, len(X_train.columns)):
    if i % 10 == 0:  # this helps me understand how the loop is going
        print(i)
 
    col_1 = X_train.columns[i]
 
    for col_2 in X_train.columns[i + 1:]:
        if X_train[col_1].equals(X_train[col_2]):
            duplicated_feat.append(col_2)
            
len(duplicated_feat)

In [ ]:
# Remove duplicated features


X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
X_test.drop(labels=duplicated_feat, axis=1, inplace=True)


X_train.shape, X_test.shape
# Keep a copy of the dataset except constant and duplicated variables to measure the performance of machine learning models
X_train_basic_filter = X_train.copy()
X_test_basic_filter = X_test.copy()

X_train.shape, X_test.shape

Remove intercorrelated features

In [ ]:
# Brute Force Method to find Correlation between features
def correlation(data, threshold=None):
    # Set of all names of correlated columns
    col_corr = set()
    corr_mat = data.corr(method='spearman')
    for i in range(len(corr_mat.columns)):
        for j in range(i):
            if (abs(corr_mat.iloc[i,j]) > threshold):
                colname = corr_mat.columns[i]
                col_corr.add(colname)
    return col_corr

In [ ]:
#feel free to change the threshold number 
correlated_features = correlation(data=X_train, threshold=0.7)
len(set(correlated_features))

In [ ]:
X_train.drop(labels=correlated_features, axis=1, inplace=True)
X_test.drop(labels=correlated_features, axis=1, inplace=True)

X_train.shape, X_test.shape

#Wrapper methods 


1.   Sequential Forward Selection

2.   Sequential Backward Selection

3.   Sequential Forward Floating Selection

4. Sequential Backward Floating Selection 

for more details refer to: [mlxtend](http://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/)


In [ ]:
# Sequential Forward Selection
sfs = SFS(knn, 
          k_features=3, 
          forward=True, 
          floating=False, 
          scoring='accuracy',
          cv=4,
          n_jobs=-1)
sfs = sfs.fit(X, y)

print('\nSequential Forward Selection (k=3):')
print(sfs.k_feature_idx_)
print('CV Score:')
print(sfs.k_score_)

###################################################

# Sequential Backward Selection
sbs = SFS(knn, 
          k_features=3, 
          forward=False, 
          floating=False, 
          scoring='accuracy',
          cv=4,
          n_jobs=-1)
sbs = sbs.fit(X, y)

print('\nSequential Backward Selection (k=3):')
print(sbs.k_feature_idx_)
print('CV Score:')
print(sbs.k_score_)

###################################################

# Sequential Forward Floating Selection
sffs = SFS(knn, 
           k_features=3, 
           forward=True, 
           floating=True, 
           scoring='accuracy',
           cv=4,
           n_jobs=-1)
sffs = sffs.fit(X, y)

print('\nSequential Forward Floating Selection (k=3):')
print(sffs.k_feature_idx_)
print('CV Score:')
print(sffs.k_score_)

###################################################

# Sequential Backward Floating Selection
sbfs = SFS(knn, 
           k_features=3, 
           forward=False, 
           floating=True, 
           scoring='accuracy',
           cv=4,
           n_jobs=-1)
sbfs = sbfs.fit(X, y)

print('\nSequential Backward Floating Selection (k=3):')
print(sbfs.k_feature_idx_)
print('CV Score:')
print(sbfs.k_score_)

In [ ]:
# Which features?
feat_cols = list(sfs1.k_feature_idx_)
print(feat_cols)

In [ ]:
# Build full model with selected features

from sklearn.metrics import mean_absolute_error as acc
clf = model
clf.fit(X_train[:, feat_cols], y_train)

y_train_pred = clf.predict(X_train[:, feat_cols])
print('Training accuracy on selected features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test[:, feat_cols])
print('Testing accuracy on selected features: %.3f' % acc(y_test, y_test_pred))

In [ ]:
#if data is scaled use this 
print('Training accuracy on selected features: %.3f' %acc(ss_y.inverse_transform(y_train), ss_y.inverse_transform(y_train_pred)))

print('Training accuracy on selected features: %.3f' %acc(ss_y.inverse_transform(y_test), ss_y.inverse_transform(y_test_pred)))

In [ ]:
# Build full model on ALL features, for comparison

from sklearn.metrics import mean_absolute_error as acc

clf = model
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
print('Training accuracy on all features: %.3f' % acc(y_train, y_train_pred))

y_test_pred = clf.predict(X_test)
print('Testing accuracy on all features: %.3f' % acc(y_test, y_test_pred))

In [ ]:
#if data is scaled use this 
print('Training accuracy on selected features: %.3f' %acc(ss_y.inverse_transform(y_train), ss_y.inverse_transform(y_train_pred)))

print('Training accuracy on selected features: %.3f' %acc(ss_y.inverse_transform(y_test), ss_y.inverse_transform(y_test_pred)))